In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import datetime as pkgdt
import requests 
import geopandas
import contextily as cx
import folium
from requests.adapters import HTTPAdapter

In [2]:
r = requests.get(
    "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json",
    headers = ''
)

if r.status_code != 200: # see HTTP errors
    print("HTTP ERROR")
else:  
    r.text
    infoStations = pd.DataFrame(pd.DataFrame(r.json())["data"]["stations"]) 

In [3]:
r = requests.get(
   "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json",
    headers = ''
)

if r.status_code != 200: # see HTTP errors
    print("HTTP ERROR")
else:  
    r.text
    statusStations = pd.DataFrame(pd.DataFrame(r.json())["data"]["stations"])

In [4]:
infoStations.head()

,station_id,name,lat,lon,capacity,stationCode,rental_methods
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN
1,653222953,Mairie de Rosny-sous-Bois,48.871257,2.486581,30,31104,[CREDITCARD]
2,516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104,NaN
3,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD]
4,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN


In [5]:
statusStations["num_bikes_mechanical"] = statusStations['num_bikes_available_types'][0][0]["mechanical"]
statusStations["num_bikes_ebike"] = statusStations['num_bikes_available_types'][0][1]["ebike"]

In [6]:
for i in statusStations.index:
    statusStations.loc[i,"num_bikes_mechanical"] = statusStations.loc[i,'num_bikes_available_types'][0]["mechanical"]
    statusStations.loc[i,"num_bikes_ebike"] = statusStations.loc[i,'num_bikes_available_types'][1]["ebike"]
statusStations.drop(['num_bikes_available_types', 'stationCode'],axis=1, inplace=True)

In [7]:
statusStations.head()

,station_id,num_bikes_available,numBikesAvailable,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported,num_bikes_mechanical,num_bikes_ebike
0,213688169,6,6,29,29,1,1,1,1668535463,4,2
1,653222953,8,8,22,22,1,1,1,1668535065,2,6
2,516709288,16,16,4,4,1,1,1,1668535199,4,12
3,36255,13,13,8,8,1,1,1,1668535414,8,5
4,37815204,19,19,10,10,1,1,1,1668535505,14,5


In [8]:
statusStations.set_index("station_id", inplace=True)
infoStations.set_index("station_id", inplace=True)

In [9]:
statusStations = statusStations.join(infoStations)
statusStations.head()

,num_bikes_available,numBikesAvailable,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported,num_bikes_mechanical,num_bikes_ebike,name,lat,lon,capacity,stationCode,rental_methods
station_id,,,,,,,,,,,,,,,,
213688169,6,6,29,29,1,1,1,1668535463,4,2,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN
653222953,8,8,22,22,1,1,1,1668535065,2,6,Mairie de Rosny-sous-Bois,48.871257,2.486581,30,31104,[CREDITCARD]
516709288,16,16,4,4,1,1,1,1668535199,4,12,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104,NaN
36255,13,13,8,8,1,1,1,1668535414,8,5,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD]
37815204,19,19,10,10,1,1,1,1668535505,14,5,Mairie du 12ème,48.840855,2.387555,30,12109,NaN


In [10]:
gdf = geopandas.GeoDataFrame(statusStations, geometry=geopandas.points_from_xy(infoStations.lon, infoStations.lat))

In [11]:
gdf.set_crs(epsg=4326, inplace=True)
print(gdf.crs)

epsg:4326


In [18]:
gdf.explore("num_bikes_mechanical", 
              legend=True,            
              legend_kwds={'label': "Nombre de vélos mécaniques par station",
                        'orientation': "horizontal"})